## Create a permanent forest cover layer for MODIS 2000 - 2020

In [1]:
import ee
import geemap 
import os
ee.Initialize()

In [2]:
# again use the large ROI so I can do a more precise crop/mask/resample with gdal to match MODIS data 
roi = ee.FeatureCollection('users/kmcquil/SBR_XL_simplified').geometry()


In [6]:
# use the MODIS Annual Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
dataset = ee.ImageCollection('MODIS/006/MCD12Q1')
igbpLandCover = dataset.select('LC_Type1')
QC = dataset.select('QC')

In [ ]:
# Mask out pixels not in [1,2,3,4,5]

# create a mask for each year from the QC band  
# Keep only pixels == 0 for QC band 
# 0 corresponds to "Classified land: has a classification label and is land according to the water mask."

# Identify pixels that have the same classification for each year 


In [22]:
# Mask out pixels not in [1,2,3,4,5] in regular land cover images 
def fmask(im):
    forest_mask = im.eq(1).max(im.eq(2)).max(im.eq(3)).max(im.eq(4)).max(im.eq(5)).clip(roi)
    return im.updateMask(forest_mask) 

igbpLandCover_fmask = igbpLandCover.map(fmask)

In [94]:
# what years is this available 
idx = igbpLandCover_fmask.aggregate_array('system:index')
print('Index: '+str(idx.getInfo())) # ee.List of band names

def qc_mask(im):
    y = ee.Number.parse(ee.Date(im.get('system:time_start')).format('yyyy')).toInt() # get the year 
    qc_im = QC.filter(ee.Filter.calendarRange(y, y, 'year')).first() # use the year to grab the QC image that matches the year of the LC image
    qmask = qc_im.eq(0).clip(roi) # create a mask using the QC image that 
    return im.updateMask(qmask)


igbpLandCover_masked = igbpLandCover_fmask.map(qc_mask)

Index: ['2001_01_01', '2002_01_01', '2003_01_01', '2004_01_01', '2005_01_01', '2006_01_01', '2007_01_01', '2008_01_01', '2009_01_01', '2010_01_01', '2011_01_01', '2012_01_01', '2013_01_01', '2014_01_01', '2015_01_01', '2016_01_01', '2017_01_01', '2018_01_01', '2019_01_01']


In [99]:
# now that each year is masked to just forest pixels, make the forest pixels all equal 1 
# then sum the collection 
# pixels = 19 (1 for each year) are permanent from 2001 - 2019 

def p_mask(im):
    return(im.eq(1).max(im.eq(2)).max(im.eq(3)).max(im.eq(4)).max(im.eq(5)))

per_for_mask = igbpLandCover_masked.map(p_mask)
total_mask = per_for_mask.sum()

# get just the pixels that = 19
m19 = total_mask.eq(19)

In [30]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [100]:
igbpLandCoverVis = {
  min: 1.0,
  max: 19.0,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301' ],
}
Map.addLayer(m19, igbpLandCoverVis)

In [101]:
# export the permanent forest layer 
task_config = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'MODIS_FOREST',
        'description': 'modis_permanent_forest',
        'image': m19,
        'maxPixels':1e13, 
        'scale' : 500
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [102]:
# export the forest for every year 

geemap.ee_export_image_collection_to_drive(ee_object= igbpLandCover_masked, 
folder='MODIS_YEARLY_FOREST', 
region=roi,
crs = 'EPSG:4326', 
scale=500
)


Total number of images: 19

Exporting 2001_01_01 ...
Exporting 2002_01_01 ...
Exporting 2003_01_01 ...
Exporting 2004_01_01 ...
Exporting 2005_01_01 ...
Exporting 2006_01_01 ...
Exporting 2007_01_01 ...
Exporting 2008_01_01 ...
Exporting 2009_01_01 ...
Exporting 2010_01_01 ...
Exporting 2011_01_01 ...
Exporting 2012_01_01 ...
Exporting 2013_01_01 ...
Exporting 2014_01_01 ...
Exporting 2015_01_01 ...
Exporting 2016_01_01 ...
Exporting 2017_01_01 ...
Exporting 2018_01_01 ...
Exporting 2019_01_01 ...
